In [1]:
import pickle

import networkx as nx
import numpy as np
from rdkit import Chem

import torch
from torch_geometric.data import Data, Batch
torch.cuda.empty_cache()

from src import utils
from src.reduction.struture_attributed_reduction import get_coresed_graph, get_expanded_graph
from src.datasets.dataset_utils import to_undirect, init_single_expanded_G, expand_graphs
with open('coarsed_graphs.pkl','rb') as f:
    coarsed_graphs = pickle.load(f)

Found rdkit, all good


/home/chensm22/miniconda3/envs/graph-generation/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO

In [2]:
coarsed_graphs

[[tensor([5, 3, 0, 0, 3, 0, 0, 4, 0]),
  tensor([1, 0, 0, 0, 0, 0, 0, 0, 0]),
  tensor([[0, 1, 0, 0, 0, 0, 0, 1, 0],
          [1, 0, 1, 0, 0, 0, 1, 0, 0],
          [0, 1, 0, 0, 0, 1, 0, 0, 0],
          [0, 0, 0, 0, 0, 1, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 1, 0],
          [0, 0, 1, 1, 0, 0, 0, 0, 1],
          [0, 1, 0, 0, 0, 0, 0, 0, 1],
          [1, 0, 0, 0, 1, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 1, 1, 0, 0]])],
 [tensor([5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0]),
  tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
  tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
          [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
          [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
          [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
          [0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [3]:
expanded_batch = expand_graphs(coarsed_graphs, False)


In [4]:
expanded_batch[1].x_type_mask

tensor([3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3,
        0])

In [5]:
x_type_mask=expanded_batch.x_type_mask + 1 # encode null mask
masked_edge_index=expanded_batch.masked_edge_index
edge_type_mask=expanded_batch.edge_type_mask + 1 # encode null mask
batch = expanded_batch.batch
print(masked_edge_index)
X_type_mask, E_type_mask, node_mask = \
    utils.masks_to_dense(x_type_mask, masked_edge_index, edge_type_mask, batch, True)

AttributeError: 'list' object has no attribute 'x_type_mask'

In [13]:
from torch_geometric.utils import to_dense_adj
import torch
import numpy as np
edge_index = torch.tensor([[0, 0, 0, 1, 2, 3],
                           [0, 1, 1, 0, 3, 0]])

unique_edges,counts = np.unique(edge_index.T, axis=0, return_counts=True)

# 打印唯一边
print(counts)

# 若需要判断重复边数量
num_unique_edges = unique_edges.shape[0]
print("Unique edges count:", num_unique_edges)
batch = torch.tensor([0, 0, 1, 1])

to_dense_adj(edge_index, batch, max_num_nodes=4)

[1 2 1 1 1]
Unique edges count: 5


tensor([[[1., 2., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])